In [5]:
import asyncio
import edge_tts
import os
import nest_asyncio
# --- ส่วนที่เพิ่ม: Import MoviePy สำหรับรวมเสียง ---
from moviepy.editor import AudioFileClip, concatenate_audioclips, AudioClip

# แก้ปัญหา Event Loop ใน Jupyter
nest_asyncio.apply()

# 1. กำหนดเสียง
VOICES = {
    "narrator": "en-US-RogerNeural",      # เสียงผู้ชาย ทางการ
    "man": "en-US-ChristopherNeural",     # เสียงผู้ชาย สนทนา
    "woman": "en-US-JennyNeural"          # เสียงผู้หญิง สนทนา
}


In [6]:
#Set5
questions_data = [
    {
        "id": 1,
        "narrator": "What does the woman imply?",
        "script_eng": "(Man): I'm thinking of buying that used car we saw yesterday.\n(Woman): Are you sure? It seemed to have a lot of mileage on it.",
        "narrator_thai": "ผู้หญิงบอกเป็นนัยว่าอะไร?",
        "script_thai": "(Man): ผมกำลังคิดว่าจะซื้อรถมือสองคันที่เราไปดูเมื่อวาน\n(Woman): คุณแน่ใจเหรอ? ดูเหมือนรถคันนั้นจะเลขไมล์เยอะมากเลยนะ (วิ่งมาเยอะ)",
        "choices_eng": [
            "(A) The man should buy the car.",
            "(B) The car is in good condition.",
            "(C) The car might not be a good purchase.",
            "(D) She wants to buy the car herself."
        ],
        "choices_thai": [
            "(ผู้ชายควรซื้อรถคันนั้น)",
            "(รถอยู่ในสภาพดี)",
            "(รถคันนั้นอาจจะไม่ใช่การซื้อที่คุ้มค่านัก)",
            "(เธออยากซื้อรถคันนั้นเอง)"
        ],
        "answer_text": "เฉลย: (C)\nอธิบาย: ผู้หญิงทักท้วงว่า 'Are you sure?' และชี้ข้อเสียเรื่อง 'lot of mileage' (เลขไมล์เยอะ) เป็นนัยว่ารถสภาพไม่ดีและไม่น่าซื้อ"
    },
    {
        "id": 2,
        "narrator": "What does the man mean?",
        "script_eng": "(Woman): I'm having trouble with this calculus problem. It's so confusing.\n(Man): Don't look at me. Math was never my strong suit.",
        "narrator_thai": "ผู้ชายหมายความว่าอย่างไร?",
        "script_thai": "(Woman): ฉันกำลังงงกับโจทย์แคลคูลัสข้อนี้ มันสับสนจัง\n(Man): อย่ามองผมสิ (อย่าหวังพึ่งผม) วิชาเลขไม่เคยเป็นจุดแข็งของผมเลย",
        "choices_eng": [
            "(A) He is very good at math.",
            "(B) He can help her with the problem.",
            "(C) He doesn't like the woman.",
            "(D) He is not good at math."
        ],
        "choices_thai": [
            "(เขาเก่งเลขมาก)",
            "(เขาสามารถช่วยเธอทำโจทย์ได้)",
            "(เขาไม่ชอบผู้หญิงคนนี้)",
            "(เขาไม่เก่งเลข)"
        ],
        "answer_text": "เฉลย: (D)\nอธิบาย: สำนวน 'never my strong suit' หมายความว่า ไม่เก่งหรือไม่ถนัดเรื่องนั้นๆ"
    },
    {
        "id": 3,
        "narrator": "What will the woman probably do?",
        "script_eng": "(Man): The elevator is broken again. We'll have to take the stairs to the 10th floor.\n(Woman): You go ahead. I'll wait for the next elevator in the lobby.",
        "narrator_thai": "ผู้หญิงน่าจะทำอะไร?",
        "script_thai": "(Man): ลิฟต์เสียอีกแล้ว เราต้องเดินขึ้นบันไดไปชั้น 10 กันแล้วล่ะ\n(Woman): คุณไปก่อนเลย ฉันจะรอลิฟต์ตัวถัดไปที่ล็อบบี้",
        "choices_eng": [
            "(A) Walk up the stairs.",
            "(B) Fix the elevator.",
            "(C) Wait for another elevator.",
            "(D) Leave the building."
        ],
        "choices_thai": [
            "(เดินขึ้นบันได)",
            "(ซ่อมลิฟต์)",
            "(รอลิฟต์ตัวอื่น)",
            "(ออกจากตึก)"
        ],
        "answer_text": "เฉลย: (C)\nอธิบาย: ผู้หญิงพูดชัดเจนว่า 'I'll wait for the next elevator' (ฉันจะรอลิฟต์ตัวถัดไป)"
    },
    {
        "id": 4,
        "narrator": "What does the man suggest?",
        "script_eng": "(Woman): I left my wallet in the classroom, but the door is locked now.\n(Man): Security should have a key. Let's go find an officer.",
        "narrator_thai": "ผู้ชายแนะนำว่าอะไร?",
        "script_thai": "(Woman): ฉันลืมกระเป๋าตังค์ไว้ในห้องเรียน แต่ตอนนี้ประตูล็อคไปแล้ว\n(Man): ฝ่ายรักษาความปลอดภัยน่าจะมีกุญแจนะ เราไปหาเจ้าหน้าที่กันเถอะ",
        "choices_eng": [
            "(A) Break the door open.",
            "(B) Ask a security officer for help.",
            "(C) Leave the wallet there.",
            "(D) Call the professor."
        ],
        "choices_thai": [
            "(พังประตูเข้าไป)",
            "(ขอความช่วยเหลือจากเจ้าหน้าที่ รปภ.)",
            "(ทิ้งกระเป๋าตังค์ไว้ที่นั่น)",
            "(โทรหาอาจารย์)"
        ],
        "answer_text": "เฉลย: (B)\nอธิบาย: ผู้ชายแนะนำว่า 'Let's go find an officer' (ไปหาเจ้าหน้าที่กันเถอะ) เพื่อให้มาช่วยเปิดประตู"
    },
    {
        "id": 5,
        "narrator": "What does the woman imply about the presentation?",
        "script_eng": "(Man): How did your presentation go this morning?\n(Woman): Well, let's just say I'm glad it's over.",
        "narrator_thai": "ผู้หญิงบอกเป็นนัยเกี่ยวกับการนำเสนองานว่าอย่างไร?",
        "script_thai": "(Man): การนำเสนองานเมื่อเช้าเป็นยังไงบ้าง?\n(Woman): เอาเป็นว่า ฉันดีใจที่มันจบลงสักที",
        "choices_eng": [
            "(A) It went extremely well.",
            "(B) She wants to do it again.",
            "(C) It didn't go very well.",
            "(D) It was too short."
        ],
        "choices_thai": [
            "(มันผ่านไปได้ด้วยดีมากๆ)",
            "(เธออยากทำมันอีกครั้ง)",
            "(มันไม่ค่อยดีเท่าไหร่)",
            "(มันสั้นเกินไป)"
        ],
        "answer_text": "เฉลย: (C)\nอธิบาย: วลี 'I'm glad it's over' (ดีใจที่มันจบไปซะที) มักใช้เมื่อเหตุการณ์นั้นยากลำบากหรือไม่น่าอภิรมย์ สื่อเป็นนัยว่าผลลัพธ์ไม่ค่อยดีนัก"
    },
    {
        "id": 6,
        "narrator": "What does the man mean?",
        "script_eng": "(Woman): Do you want to try the new seafood place tonight?\n(Man): I'm not really a big fan of fish.",
        "narrator_thai": "ผู้ชายหมายความว่าอย่างไร?",
        "script_thai": "(Woman): คืนนี้คุณอยากไปลองร้านอาหารทะเลเปิดใหม่ไหม?\n(Man): ผมไม่ค่อยชอบกินปลาเท่าไหร่นะ",
        "choices_eng": [
            "(A) He loves seafood.",
            "(B) He wants to go to the restaurant.",
            "(C) He is a fan of the restaurant.",
            "(D) He prefers not to eat seafood."
        ],
        "choices_thai": [
            "(เขาชอบอาหารทะเลมาก)",
            "(เขาอยากไปที่ร้านอาหารนั้น)",
            "(เขาเป็นแฟนคลับของร้านนั้น)",
            "(เขาเลือกที่จะไม่กินอาหารทะเล)"
        ],
        "answer_text": "เฉลย: (D)\nอธิบาย: สำนวน 'not a big fan of...' แปลว่า ไม่ค่อยชอบ ดังนั้นเขาจึงไม่อยากไปกิน (Prefers not to eat)"
    },
    {
        "id": 7,
        "narrator": "What does the woman suggest the man do?",
        "script_eng": "(Man): I've got a terrible headache, and I have a chemistry lab in an hour.\n(Woman): If I were you, I’d take some aspirin and lie down for a bit.",
        "narrator_thai": "ผู้หญิงแนะนำให้ผู้ชายทำอะไร?",
        "script_thai": "(Man): ผมปวดหัวจี๊ดเลย แล้วมีเรียนแล็บเคมีในอีกชั่วโมงนึงด้วย\n(Woman): ถ้าฉันเป็นคุณ ฉันจะกินยาแอสไพรินแล้วนอนพักสักหน่อย",
        "choices_eng": [
            "(A) Go to the chemistry lab immediately.",
            "(B) Take medicine and rest.",
            "(C) Study for the lab.",
            "(D) See a doctor."
        ],
        "choices_thai": [
            "(ไปเข้าแล็บเคมีทันที)",
            "(กินยาและพักผ่อน)",
            "(อ่านหนังสือเตรียมเข้าแล็บ)",
            "(ไปหาหมอ)"
        ],
        "answer_text": "เฉลย: (B)\nอธิบาย: คำแนะนำคือ 'take some aspirin' (กินยา) และ 'lie down' (นอนพัก/rest)"
    },
    {
        "id": 8,
        "narrator": "What implies about the assignment?",
        "script_eng": "(Woman): Have you started writing your essay yet?\n(Man): Started it? I handed it in two days ago!",
        "narrator_thai": "บทสนทนาบอกอะไรเกี่ยวกับงานเขียน?",
        "script_thai": "(Woman): คุณเริ่มเขียนเรียงความหรือยัง?\n(Man): เริ่มเหรอ? ผมส่งงานไปตั้งแต่ 2 วันที่แล้วแล้ว!",
        "choices_eng": [
            "(A) The man hasn't started it yet.",
            "(B) The man is writing it now.",
            "(C) The man finished it early.",
            "(D) The man lost the essay."
        ],
        "choices_thai": [
            "(ผู้ชายยังไม่ได้เริ่มทำ)",
            "(ผู้ชายกำลังเขียนอยู่)",
            "(ผู้ชายทำเสร็จก่อนกำหนด)",
            "(ผู้ชายทำเรียงความหาย)"
        ],
        "answer_text": "เฉลย: (C)\nอธิบาย: ผู้ชายบอกว่า 'handed it in two days ago' (ส่งไป 2 วันแล้ว) แสดงว่าเขาทำเสร็จเร็ว (Finished early) ก่อนที่ผู้หญิงจะถามเสียอีก"
    },
    {
        "id": 9,
        "narrator": "What does the woman mean?",
        "script_eng": "(Man): I can't believe we have to buy three textbooks for this course. They are so expensive.\n(Woman): Tell me about it. My wallet is empty.",
        "narrator_thai": "ผู้หญิงหมายความว่าอย่างไร?",
        "script_thai": "(Man): ไม่อยากเชื่อเลยว่าเราต้องซื้อหนังสือเรียนตั้ง 3 เล่มสำหรับวิชานี้ แพงชะมัด\n(Woman): เห็นด้วยสุดๆ กระเป๋าตังค์ฉันเกลี้ยงแล้วเนี่ย",
        "choices_eng": [
            "(A) She has lost her wallet.",
            "(B) She disagrees with the man.",
            "(C) She thinks the books are cheap.",
            "(D) She agrees that the books are expensive."
        ],
        "choices_thai": [
            "(เธอทำกระเป๋าตังค์หาย)",
            "(เธอไม่เห็นด้วยกับผู้ชาย)",
            "(เธอคิดว่าหนังสือราคาถูก)",
            "(เธอเห็นด้วยว่าหนังสือแพง)"
        ],
        "answer_text": "เฉลย: (D)\nอธิบาย: วลี 'Tell me about it' เป็นการแสดงความเห็นด้วยอย่างยิ่ง และการบอกว่า 'wallet is empty' ย้ำว่าหนังสือแพงจนเงินหมด"
    },
    {
        "id": 10,
        "narrator": "What will the man probably do?",
        "script_eng": "(Woman): The library is closing in five minutes. You'd better check those books out now.\n(Man): Oh, thanks for the warning. I'll head to the front desk right away.",
        "narrator_thai": "ผู้ชายน่าจะทำอะไร?",
        "script_thai": "(Woman): ห้องสมุดจะปิดในอีก 5 นาทีนะ คุณรีบไปยืมหนังสือพวกนั้นเดี๋ยวนี้เลยดีกว่า\n(Man): โอ้ ขอบคุณที่เตือน ผมจะรีบไปที่เคาน์เตอร์เดี๋ยวนี้เลย",
        "choices_eng": [
            "(A) Leave the books on the table.",
            "(B) Go to the front desk to borrow books.",
            "(C) Complain to the librarian.",
            "(D) Continue reading."
        ],
        "choices_thai": [
            "(ทิ้งหนังสือไว้บนโต๊ะ)",
            "(ไปที่เคาน์เตอร์เพื่อยืมหนังสือ)",
            "(ไปบ่นกับบรรณารักษ์)",
            "(อ่านหนังสือต่อ)"
        ],
        "answer_text": "เฉลย: (B)\nอธิบาย: ผู้ชายตอบรับว่าจะ 'head to the front desk' (มุ่งหน้าไปที่เคาน์เตอร์) เพื่อทำเรื่องยืมหนังสือ (check out)"
    }
]

In [7]:
# 3. ฟังก์ชันแปลงข้อมูลดิบเป็น Script Segments
def build_script_segments(question_data):
    segments = []
    
    # 3.1 Intro
    segments.append({
        "role": "narrator",
        "text": f"Question number {question_data['id']}."
    })
    
    # 3.2 Dialog
    lines = question_data['script_eng'].split('\n')
    for line in lines:
        line = line.strip()
        if not line: continue
        
        if line.lower().startswith("(man):"):
            role = "man"
            text = line[6:].strip()
        elif line.lower().startswith("(woman):"):
            role = "woman"
            text = line[8:].strip()
        else:
            continue
            
        segments.append({"role": role, "text": text})
    
    # 3.3 Question Text
    segments.append({
        "role": "narrator",
        "text": question_data['narrator']
    })
    
    # 3.4 Choices
    for choice in question_data['choices_eng']:
        segments.append({
            "role": "narrator",
            "text": choice
        })
        
    return segments

# --- ส่วนที่เพิ่ม: ฟังก์ชันสร้างเสียงเงียบ ---
def make_silence(duration):
    return AudioClip(lambda t: [0], duration=duration, fps=44100)

# --- ส่วนที่เพิ่ม: ฟังก์ชันรวมไฟล์เสียง ---
def merge_audio_in_folder(folder_path, output_filename):
    # หาไฟล์ .mp3 ที่ขึ้นต้นด้วยตัวเลข (01_, 02_...)
    files = sorted([f for f in os.listdir(folder_path) if f.endswith(".mp3") and f[0].isdigit()])
    
    if not files:
        return

    clips = []
    silence = make_silence(1.0) # เสียงเงียบ 1 วินาที คั่นระหว่างประโยค

    for i, filename in enumerate(files):
        path = os.path.join(folder_path, filename)
        try:
            clip = AudioFileClip(path)
            clips.append(clip)
            
            # เพิ่มเสียงเงียบ ถ้าไม่ใช่ไฟล์สุดท้าย
            if i < len(files) - 1:
                clips.append(silence)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

    # รวมไฟล์
    if clips:
        final_clip = concatenate_audioclips(clips)
        final_path = os.path.join(folder_path, output_filename)
        final_clip.write_audiofile(final_path, verbose=False, logger=None)
        print(f"   ✨ รวมไฟล์สำเร็จ: {final_path}")

# 4. ฟังก์ชันสร้างเสียงหลัก
async def generate_all_questions():
    base_folder = "TOEFL_Audio_Output"
    if not os.path.exists(base_folder):
        os.makedirs(base_folder)
        
    print(f"🚀 เริ่มสร้างไฟล์เสียงสำหรับ {len(questions_data)} ข้อ...\n")

    for q_data in questions_data:
        q_id = q_data['id']
        folder_name = os.path.join(base_folder, f"Q{q_id:02d}") 
        
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
            
        segments = build_script_segments(q_data)
        
        print(f"🔹 กำลังสร้างเสียงข้อที่ {q_id}...")
        
        # 4.1 สร้างไฟล์ย่อย (01, 02, 03...)
        for idx, seg in enumerate(segments):
            role = seg['role']
            text = seg['text']
            voice = VOICES[role]
            
            filename = f"{idx+1:02d}_{role}.mp3"
            output_path = os.path.join(folder_name, filename)
            
            communicate = edge_tts.Communicate(text, voice)
            await communicate.save(output_path)
            
        # 4.2 รวมไฟล์ (เรียกฟังก์ชันที่เพิ่มเข้ามา)
        merge_audio_in_folder(folder_name, f"Full_Q{q_id:02d}.mp3")

    print(f"\n✅ เสร็จสมบูรณ์! ไฟล์ทั้งหมดอยู่ที่โฟลเดอร์: {base_folder}")
    # os.system(f"start {base_folder}")

# 5. รันโปรแกรม
loop = asyncio.get_event_loop()
loop.run_until_complete(generate_all_questions())

🚀 เริ่มสร้างไฟล์เสียงสำหรับ 10 ข้อ...

🔹 กำลังสร้างเสียงข้อที่ 1...
   ✨ รวมไฟล์สำเร็จ: TOEFL_Audio_Output\Q01\Full_Q01.mp3
🔹 กำลังสร้างเสียงข้อที่ 2...
   ✨ รวมไฟล์สำเร็จ: TOEFL_Audio_Output\Q02\Full_Q02.mp3
🔹 กำลังสร้างเสียงข้อที่ 3...
   ✨ รวมไฟล์สำเร็จ: TOEFL_Audio_Output\Q03\Full_Q03.mp3
🔹 กำลังสร้างเสียงข้อที่ 4...
   ✨ รวมไฟล์สำเร็จ: TOEFL_Audio_Output\Q04\Full_Q04.mp3
🔹 กำลังสร้างเสียงข้อที่ 5...
   ✨ รวมไฟล์สำเร็จ: TOEFL_Audio_Output\Q05\Full_Q05.mp3
🔹 กำลังสร้างเสียงข้อที่ 6...
   ✨ รวมไฟล์สำเร็จ: TOEFL_Audio_Output\Q06\Full_Q06.mp3
🔹 กำลังสร้างเสียงข้อที่ 7...
   ✨ รวมไฟล์สำเร็จ: TOEFL_Audio_Output\Q07\Full_Q07.mp3
🔹 กำลังสร้างเสียงข้อที่ 8...
   ✨ รวมไฟล์สำเร็จ: TOEFL_Audio_Output\Q08\Full_Q08.mp3
🔹 กำลังสร้างเสียงข้อที่ 9...
   ✨ รวมไฟล์สำเร็จ: TOEFL_Audio_Output\Q09\Full_Q09.mp3
🔹 กำลังสร้างเสียงข้อที่ 10...
   ✨ รวมไฟล์สำเร็จ: TOEFL_Audio_Output\Q10\Full_Q10.mp3

✅ เสร็จสมบูรณ์! ไฟล์ทั้งหมดอยู่ที่โฟลเดอร์: TOEFL_Audio_Output
